In [0]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from utils.ingestUtility import *
import yfinance as yf
import pandas as pd

In [0]:
# manual stock ingest

symbol = "PLTR"
data = yf.download([symbol], period='5y')
if isinstance(data.columns, pd.MultiIndex):  
    data.columns = data.columns.droplevel(0) 

data.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
data.reset_index(inplace=True)
data = data[['Date', 'Close', 'High', 'Low', 'Open', 'Volume']]
print(data.head())
df = spark.createDataFrame(data)
df = df.withColumn("Ticker", F.lit(f"{symbol}"))
df = df.withColumn("date", to_date(col("Date"), "yyyy-MM-dd"))

df.display()
writeToDataLake(df, f"/mnt/data/Historical/raw/{symbol}_data", "overwrite")
renameOutputFile(f"/mnt/data/Historical/raw/{symbol}_data")

In [0]:
df_delta = csvReadNoSchema(f"/mnt/data/Historical/raw/{symbol}_data")
df_delta.write.format("delta").mode("append").saveAsTable("pprtrading.stocks_historical")